In [1]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import urllib.request
import os
import urllib.parse
import shutil

import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np


In [2]:
fakeBlogKeywordList = ["스토리앤","seoulouba","mateb.kr","revu","weble","ohmyblog","mrblog","tble",
    "dinnerqueen","%EA%B3%B5%EC%A0%95%EA%B1%B0%EB%9E%98%EC%9C%84%EC%9B%90%ED%9A%8C-%EB%AC%B8%EA%B5%AC",
    "%EB%B8%94%EB%A1%9C%EA%B7%B8%EC%9B%90%EC%A0%95%EB%8C%80",
    "banner_","%EC%9D%B4%EC%8A%88%EB%B8%94%EB%A1%9C%EA%B7%B8",
    "%EB%B0%B0%EB%84%88","http://echoblog.net/images/sponsor-banner.png",
                      "sponsor","banner","echoblog"]

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

#define const value
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150


In [4]:
def create_model():
    model = Sequential([
        Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
        MaxPooling2D(),
        Conv2D(32, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Conv2D(64, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [5]:
#load model
checkpoint_path = "trained_advertise_0616/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

# create model
model = create_model()
model.load_weights(latest)

data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
teachable_model = tensorflow.keras.models.load_model('/Users/esens/Downloads/converted_keras/keras_model.h5')
np.set_printoptions(suppress=True)

trained_advertise_0616/cp-0015.ckpt


In [22]:
def teachableMachinePrediction(image_path):
    image = Image.open(image_path)
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    newdata = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)    
    
    if(normalized_image_array.shape == (224,224,4)):
        print('224,224,4 : ' , image_path)
        print(normalized_image_array.shape)
        normalized_image_array_reshape = normalized_image_array.reshape(224,224,3)
        print(normalized_image_array_reshape.shape)

    elif (normalized_image_array.shape == (224, 224)):
        print('224,224 : ', image_path)
        normalized_image_array_reshape = normalized_image_array.reshape(-1,224,224,3)
        print(normalized_image_array_reshape.shape)
        
    else:
        normalized_image_array_reshape = normalized_image_array.reshape(1,224,224,3)

    # Load the image into the array
    newdata[0] = normalized_image_array_reshape
    print('newdata.shape : ' , newdata.shape)
    print('newdata : ' , newdata)
    #predictions class 
    # {0 : advertise , 1: others}
    # run the inference
    
#     image = tf.keras.preprocessing.image.load_img(image_path,target_size=(224, 224))
#     input_arr = tf.keras.preprocessing.image.img_to_array(image)
#     input_arr = np.array([input_arr])
    
    prediction = teachable_model.predict(newdata)
    prediction_class = teachable_model.predict_classes(newdata)
    #print("========pridiction[teachable]========")
    #print(prediction_class)
    return prediction_class

def imageClassification(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path,target_size=(150, 150))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])  # Convert single image to a batch.

    #print("model shape : " , model.summary());
    # print("input_arr : ", input_arr)

    predictions_class = model.predict_classes(input_arr)
    predictions = model.predict(input_arr)

#    print("========pridiction========")
#     print(test_data_gen.class_indices)
#    print("predictions class : " , predictions_class);
#    print("predictions_class : " , predictions_class);
#    print("predictions argmax : ",np.argmax(predictions[0])); # same way
    return predictions_class;

#이미지 저장
def saveImage(div_option,index,img_url,nickName,postNumber):
    #블로그 유저 별로 폴더를 나눌건지에 대한 옵션
    #true -> path : image/{nickName}/{postNumber}
    #false -> path : image/myimg/
    if(div_option):
        if nickName is "":
            nickName = "unknown"
        if postNumber is "":
            postNumber = "1"        
        imgPath = f'image/{nickName}/{postNumber}/{index}.jpg'
        dirPath = f'image/{nickName}/{postNumber}'
    #폴더별로 분기하지 않을떄 path : image/myimg/
    else:
        list = os.listdir("image/temp")
        last_index = len(list)
#         print("last_index : ",last_index)
        dirPath = 'image/temp'
        imgPath = f'image/temp/{last_index}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
                
    #파일 저장
    #temp에 저장 -> classification -> 분류 저장
    
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)

        for item in fakeBlogKeywordList:
            if(img_url_final.find(item) > 0):
                print('this is fake img')

                imgPath = f"image/temp/fake_{last_index}.jpg"
                break;
        
        urllib.request.urlretrieve(img_url_final, imgPath)
        #분류
        #{'advertise': 0, 'image': 1}
        #my_prediction  = imageClassification(imgPath);
        my_prediction  = teachableMachinePrediction(imgPath);
        print("prediction : ", my_prediction[0]);
        
        if (my_prediction[0] == 0):
            list = os.listdir("ad_img")
            last_index = len(list)
            #os.rename(imgPath, f'ad_img/{last_index}.jpg')
            #shutil.move(imgPath, f'ad_img/{last_index}.jpg')
            os.replace(imgPath, f'ad_img/{last_index}.jpg')
            return False
        return True
    except Exception as ex:
        print("img save error : " , ex)
        return False
    
def findBlogImgGethering(searchQuery, searchOption, page):
    url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
    html = requests.get(url)
    # 1차, blog URL 찾기
    soup = BeautifulSoup(html.text, 'html.parser')
    index = 0;

    url_list = [];
    for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
        child_item = li_item.find('a',{'class' : 'sh_blog_title'})
        title = child_item.attrs['title']
        href = child_item.attrs['href']

        #url list 저장
        url_list.append(href)

        print(f'title : {title} , href : {href}')
        print('----------------------------------')
        
    # url 파싱 및 예외처리
    for url_item in url_list:
        nickName = "";
        postNumber = "";

        if(url_item.find("blog.me") > 0):
            parsing = url_item;
            parsing = parsing.replace("https://","")
            nickName = parsing.split('.')[0]
            postNumber = parsing.split('/')[1]
            blogUrl = "https://m.blog.naver.com/" + nickName + "?Redirect=Log&logNo=" + postNumber

        else:
            blogUrl = url_item.replace("https://", "https://m.");

        #https://m.blog.naver.com/soeyoung1004?Redirect=Log&logNo=221944814459

        nickName = blogUrl.split('/')[-1].split('?')[0]
        postNumber = blogUrl.split('/')[-1].split('?')[1].split('=')[-1]

        if nickName is "":
            nickName = "unknown"
        if postNumber is "":
            postNumber = "1"
            
        print("------ digging more -------")
        print("nickName : " , blogUrl.split('/')[-1].split('?')[0])
        print("postNumber : " , blogUrl.split('/')[-1].split('?')[1].split('=')[-1])
        print("blogUrl : " , blogUrl)        

        #전체 목록 순회
        blog_html = requests.get(blogUrl)
        blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
        blog_image_class = blog_soup.find_all('div',{'class' : 'se-image'});

        for div_obj in blog_image_class:
            for idx, img_item in enumerate(div_obj.find_all('img')):
                img_url = img_item.attrs['src'];

                if(img_url.find("w80_blur") > 0):
                    img_url = img_url.replace("w80_blur" , "w800")
                isSaveSuccess = saveImage(False,index,img_url,nickName,postNumber)
                if(isSaveSuccess):  
                    index = index+1
#                    print('index  : ' , index , "blogUrl : " + blogUrl)
#                 if not os.path.isdir(f'image/{nickName}/{postNumber}'):
#                     try:
#                         os.makedirs(f'image/{nickName}/{postNumber}')

#                     except Exception as ex:
#                         print("error catch : " , ex)
#                 try:
#                     filename = img_url.split('/')[-1].split('?')[0];
#                     img_url2 = urllib.parse.quote_plus(str(filename))
#                     img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
#                     urllib.request.urlretrieve(img_url_final, f'image/{nickName}/{postNumber}/{index}.jpg')
#                     index = index +1;
#                 except Exception as ex:
#                     print("img save error : " , ex)


In [23]:
searchQuery = "서울대입구역 하남돼지"
searchOption = 'sim'  #sim or date  
page = 1

findBlogImgGethering(searchQuery,searchOption,page)

title : 서울대입구역 고기집_하남돼지집 , href : https://blog.naver.com/smilly07?Redirect=Log&logNo=221986387337
----------------------------------
title : [서울대입구역] 하남돼지집 - 서울대입구역 삼겹살 , href : https://blog.naver.com/juhun000?Redirect=Log&logNo=221944015147
----------------------------------
title : 서울대입구역 삼겹살은 역시 하남돼지집 , href : https://blog.naver.com/dbwlsdbdus?Redirect=Log&logNo=221984816209
----------------------------------
title : 서울대입구역 맛집 손도 입도 편한 고기꿀맛집 하남돼지집 , href : https://blog.naver.com/clmusicdi?Redirect=Log&logNo=221834473267
----------------------------------
title : 서울대입구역 삼겹살 - 하남돼지집 서울대입구점 , href : https://rich4707.blog.me/221479303167
----------------------------------
title : 서울대입구역 삼겹살 맛집 : 하남돼지집 , href : https://maxsunao.blog.me/221443369975
----------------------------------
title : 서울대입구 회식으로 유명한 하남돼지집에서 서울대입구역 삼겹살 먹어보기 , href : https://blog.naver.com/ainetquf123?Redirect=Log&logNo=221718971993
----------------------------------
title : 서울대입구역 고기집 1등은 삼겹살이 맛있는 하남돼지집 , href : htt

prediction :  1
newdata.shape :  (1, 224, 224, 3)
newdata :  [[[[-0.8897638  -0.8897638  -0.8897638 ]
   [-0.8897638  -0.8897638  -0.8897638 ]
   [-0.88188976 -0.88188976 -0.8897638 ]
   ...
   [-0.7480315  -0.7322835  -0.7322835 ]
   [-0.7480315  -0.7322835  -0.7401575 ]
   [-0.7480315  -0.7322835  -0.7401575 ]]

  [[-0.8897638  -0.8897638  -0.9055118 ]
   [-0.88188976 -0.88188976 -0.8976378 ]
   [-0.87401575 -0.87401575 -0.8897638 ]
   ...
   [-0.7401575  -0.72440946 -0.71653545]
   [-0.7322835  -0.72440946 -0.71653545]
   [-0.7401575  -0.7322835  -0.71653545]]

  [[-0.88188976 -0.88188976 -0.8976378 ]
   [-0.87401575 -0.87401575 -0.8897638 ]
   [-0.87401575 -0.87401575 -0.8897638 ]
   ...
   [-0.71653545 -0.70866144 -0.6929134 ]
   [-0.71653545 -0.70866144 -0.6929134 ]
   [-0.7322835  -0.72440946 -0.70866144]]

  ...

  [[-0.6771654  -0.62204725 -0.5511811 ]
   [-0.6850394  -0.6062992  -0.5669291 ]
   [-0.5669291  -0.46456695 -0.4094488 ]
   ...
   [-0.8346457  -0.8110236  -0.779527

newdata.shape :  (1, 224, 224, 3)
newdata :  [[[[-0.7401575  -0.54330707 -0.503937  ]
   [-0.7401575  -0.54330707 -0.503937  ]
   [-0.7322835  -0.53543305 -0.496063  ]
   ...
   [-0.71653545 -0.51968503 -0.46456695]
   [-0.70866144 -0.511811   -0.45669293]
   [-0.70866144 -0.511811   -0.45669293]]

  [[-0.7401575  -0.54330707 -0.503937  ]
   [-0.7401575  -0.54330707 -0.503937  ]
   [-0.7401575  -0.54330707 -0.503937  ]
   ...
   [-0.7322835  -0.53543305 -0.48031497]
   [-0.71653545 -0.51968503 -0.46456695]
   [-0.70866144 -0.511811   -0.45669293]]

  [[-0.7401575  -0.54330707 -0.511811  ]
   [-0.7401575  -0.54330707 -0.511811  ]
   [-0.7401575  -0.54330707 -0.511811  ]
   ...
   [-0.7559055  -0.54330707 -0.48818898]
   [-0.7401575  -0.52755904 -0.47244096]
   [-0.72440946 -0.511811   -0.45669293]]

  ...

  [[-0.6062992  -0.35433072 -0.37795275]
   [-0.6062992  -0.35433072 -0.37795275]
   [-0.6062992  -0.35433072 -0.37795275]
   ...
   [-0.6062992  -0.42519683 -0.48031497]
   [-0.63779

prediction :  1
newdata.shape :  (1, 224, 224, 3)
newdata :  [[[[-0.92913383 -0.92913383 -0.9133858 ]
   [-0.92913383 -0.92913383 -0.9133858 ]
   [-0.9370079  -0.9370079  -0.9212598 ]
   ...
   [ 0.65354335  0.29133856 -0.10236222]
   [ 0.65354335  0.3070866  -0.07874018]
   [ 0.65354335  0.29921257 -0.08661419]]

  [[-0.9370079  -0.9370079  -0.9212598 ]
   [-0.9370079  -0.9370079  -0.9212598 ]
   [-0.9448819  -0.9448819  -0.92913383]
   ...
   [ 0.64566934  0.28346455 -0.11811024]
   [ 0.65354335  0.29921257 -0.0944882 ]
   [ 0.66141737  0.29921257 -0.0944882 ]]

  [[-0.9527559  -0.9527559  -0.9370079 ]
   [-0.9527559  -0.9527559  -0.9370079 ]
   [-0.9527559  -0.9527559  -0.9370079 ]
   ...
   [ 0.64566934  0.25984251 -0.14960629]
   [ 0.64566934  0.26771653 -0.14173228]
   [ 0.64566934  0.25984251 -0.14960629]]

  ...

  [[-0.4015748  -0.8976378  -0.984252  ]
   [-0.39370078 -0.8976378  -0.984252  ]
   [-0.39370078 -0.8976378  -0.992126  ]
   ...
   [-0.96850395 -0.97637796 -0.992126

prediction :  1
newdata.shape :  (1, 224, 224, 3)
newdata :  [[[[-0.992126   -0.992126   -1.        ]
   [-0.992126   -0.992126   -1.        ]
   [-0.992126   -0.992126   -1.        ]
   ...
   [ 0.7559055   0.33858263 -0.29133856]
   [ 0.77165353  0.36220467 -0.27559054]
   [ 0.77165353  0.38582683 -0.2519685 ]]

  [[-0.992126   -0.992126   -1.        ]
   [-0.992126   -0.992126   -1.        ]
   [-0.992126   -0.992126   -1.        ]
   ...
   [ 0.7480315   0.33070862 -0.3070866 ]
   [ 0.77165353  0.36220467 -0.27559054]
   [ 0.77165353  0.38582683 -0.2519685 ]]

  [[-0.992126   -0.992126   -1.        ]
   [-0.992126   -0.992126   -1.        ]
   [-0.992126   -0.992126   -1.        ]
   ...
   [ 0.7401575   0.33070862 -0.33070868]
   [ 0.7637795   0.36220467 -0.29133856]
   [ 0.77165353  0.38582683 -0.2519685 ]]

  ...

  [[-0.86614174 -0.9448819  -1.        ]
   [-0.86614174 -0.9448819  -1.        ]
   [-0.8582677  -0.9448819  -1.        ]
   ...
   [-0.92913383 -0.92913383 -0.952755

...

prediction :  1
newdata.shape :  (1, 224, 224, 3)
newdata :  [[[[-0.51968503 -0.47244096 -0.45669293]
   [-0.5748031  -0.52755904 -0.52755904]
   [-0.6771654  -0.62992126 -0.62992126]
   ...
   [-0.8897638  -0.8503937  -0.8897638 ]
   [-0.9133858  -0.87401575 -0.9212598 ]
   [-0.9133858  -0.87401575 -0.9212598 ]]

  [[-0.62992126 -0.59055114 -0.5669291 ]
   [-0.6771654  -0.62992126 -0.62204725]
   [-0.6850394  -0.6456693  -0.6456693 ]
   ...
   [-0.8897638  -0.8582677  -0.8897638 ]
   [-0.8897638  -0.8582677  -0.9055118 ]
   [-0.9212598  -0.88188976 -0.92913383]]

  [[-0.6062992  -0.54330707 -0.51968503]
   [-0.6062992  -0.5590551  -0.54330707]
   [-0.6850394  -0.6535433  -0.63779527]
   ...
   [-0.8897638  -0.87401575 -0.9055118 ]
   [-0.88188976 -0.86614174 -0.8976378 ]
   [-0.87401575 -0.86614174 -0.8976378 ]]

  ...

  [[-0.5669291  -0.48031497 -0.53543305]
   [-0.07086617  0.02362204 -0.03937006]
   [ 0.26771653  0.35433066  0.29133856]
   ...
   [-0.03937006 -0.02362204 -0.275590

newdata.shape :  (1, 224, 224, 3)
newdata :  [[[[ 0.29133856  0.02362204 -0.17322832]
   [ 0.29133856  0.02362204 -0.17322832]
   [ 0.29921257  0.03149605 -0.17322832]
   ...
   [-0.08661419 -0.37795275 -0.59055114]
   [-0.07874018 -0.37007874 -0.5748031 ]
   [-0.11023623 -0.38582677 -0.59055114]]

  [[ 0.29921257  0.03149605 -0.16535431]
   [ 0.3070866   0.03937006 -0.16535431]
   [ 0.3070866   0.03937006 -0.17322832]
   ...
   [-0.08661419 -0.37795275 -0.5826771 ]
   [-0.08661419 -0.37795275 -0.5826771 ]
   [-0.0944882  -0.37795275 -0.5826771 ]]

  [[ 0.3070866   0.03937006 -0.17322832]
   [ 0.3149606   0.04724407 -0.16535431]
   [ 0.3149606   0.04724407 -0.16535431]
   ...
   [-0.07874018 -0.37007874 -0.5826771 ]
   [-0.0944882  -0.37795275 -0.59055114]
   [-0.07874018 -0.36220473 -0.5748031 ]]

  ...

  [[ 0.3228346   0.03149605 -0.18110234]
   [ 0.3149606   0.01574802 -0.19685042]
   [ 0.29921257 -0.01574802 -0.22047246]
   ...
   [-0.11811024 -0.4015748  -0.5984252 ]
   [-0.08661

newdata.shape :  (1, 224, 224, 3)
newdata :  [[[[-0.8503937  -0.9527559  -1.        ]
   [-0.8346457  -0.9527559  -1.        ]
   [-0.8188976  -0.9448819  -1.        ]
   ...
   [ 0.65354335  0.59055114  0.41732287]
   [ 0.54330707  0.46456695  0.28346455]
   [ 0.33858263  0.21259844  0.02362204]]

  [[-0.8582677  -0.9448819  -1.        ]
   [-0.8346457  -0.9448819  -1.        ]
   [-0.8346457  -0.9527559  -1.        ]
   ...
   [ 0.7007874   0.64566934  0.48031497]
   [ 0.65354335  0.59055114  0.41732287]
   [ 0.5511811   0.44881892  0.26771653]]

  [[-0.87401575 -0.96062994 -1.        ]
   [-0.8503937  -0.9527559  -1.        ]
   [-0.8425197  -0.9527559  -1.        ]
   ...
   [ 0.7007874   0.65354335  0.48818898]
   [ 0.64566934  0.60629916  0.4409449 ]
   [ 0.6141732   0.5511811   0.36220467]]

  ...

  [[-0.5748031  -0.8425197  -0.97637796]
   [-0.5826771  -0.8267716  -0.9448819 ]
   [-0.5826771  -0.8188976  -0.9212598 ]
   ...
   [ 0.40157485  0.04724407 -0.25984251]
   [ 0.45669

newdata.shape :  (1, 224, 224, 3)
newdata :  [[[[-0.6929134  -0.5984252  -0.52755904]
   [-0.8425197  -0.77165353 -0.6929134 ]
   [-0.9055118  -0.8425197  -0.7480315 ]
   ...
   [-0.86614174 -0.8425197  -0.72440946]
   [-0.7401575  -0.72440946 -0.63779527]
   [-0.45669293 -0.4409449  -0.4173228 ]]

  [[-0.66141737 -0.5748031  -0.496063  ]
   [-0.8346457  -0.7637795  -0.6850394 ]
   [-0.9055118  -0.8425197  -0.7480315 ]
   ...
   [-0.87401575 -0.8346457  -0.70866144]
   [-0.7322835  -0.70866144 -0.61417323]
   [-0.45669293 -0.42519683 -0.4015748 ]]

  [[-0.66141737 -0.5669291  -0.496063  ]
   [-0.8188976  -0.7559055  -0.6692914 ]
   [-0.8897638  -0.8267716  -0.7322835 ]
   ...
   [-0.88188976 -0.8267716  -0.7007874 ]
   [-0.7401575  -0.6929134  -0.6062992 ]
   [-0.44881892 -0.4094488  -0.38582677]]

  ...

  [[-0.88188976 -0.88188976 -0.88188976]
   [-0.88188976 -0.87401575 -0.88188976]
   [-0.8976378  -0.8897638  -0.8897638 ]
   ...
   [-0.9055118  -0.9055118  -0.8976378 ]
   [-0.89763

KeyboardInterrupt: 